https://pypi.org/project/arxiv/

Turns out there's a python wrapper for the arXiv API

In [6]:
!pip install --upgrade --quiet arxiv
!pip install --upgrade --quiet pymupdf

In [8]:
import arxiv

In [5]:
# Construct the default API client.
client = arxiv.Client()

# Search for the 10 most recent articles matching the keyword "quantum."
search = arxiv.Search(
  query = "quantum",
  max_results = 10,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

results = client.results(search)

# `results` is a generator; you can iterate over its elements one by one...
for r in client.results(search):
  print(r.title)
# ...or exhaust it into a list. Careful: this is slow for large results sets.
all_results = list(results)
print([r.title for r in all_results])

# For advanced query syntax documentation, see the arXiv API User Manual:
# https://arxiv.org/help/api/user-manual#query_details
search = arxiv.Search(query = "au:del_maestro AND ti:checkerboard")
first_result = next(client.results(search))
print(first_result)

# Search for the paper with ID "1605.08386v1"
search_by_id = arxiv.Search(id_list=["1605.08386v1"])
# Reuse client to fetch the paper, then print its title.
first_result = next(client.results(search))
print(first_result.title)

Mean-field theory of 1+1D $\mathbb{Z}_2$ lattice gauge theory with matter
Pair production in rotating electric fields via quantum kinetic equations: Resolving helicity states
Gaussian Process Regression with Soft Inequality and Monotonicity Constraints
Dark energy as a geometrical effect in geodetic brane gravity
Forecasting the sensitivity of Pulsar Timing Arrays to gravitational wave backgrounds
Scalable quantum detector tomography by high-performance computing
Efficient Quantum Circuits for Non-Unitary and Unitary Diagonal Operators with Space-Time-Accuracy trade-offs
Thermodynamics and Quasinormal Modes of the Dymnikova Black Hole in Higher Dimensions
Anyonic quantum multipartite maskers in the Kitaev model
Dancing above the abyss: Environmental effects and dark matter signatures in inspirals into massive black holes
['Mean-field theory of 1+1D $\\mathbb{Z}_2$ lattice gauge theory with matter', 'Pair production in rotating electric fields via quantum kinetic equations: Resolving he

Not that impressive, we already have this functionality. But we could use this later to clean up some code I wrote earlier in MVP.ipynb

Only interesting feature is the Result.download_pdf() function. Maybe we could add this feature so the user can download a preprint they find interesting

---

Langchain is probably the best way of turning this tool 'conversational'

In [17]:
# !pip install --upgrade langchain

In [12]:
from langchain_community.document_loaders import ArxivLoader

In [14]:
docs = ArxivLoader(query="quantum computing", load_max_docs=2).load()
len(docs)

2

In [15]:
docs[0].metadata  # meta-information of the Document

{'Published': '2022-08-01',
 'Title': 'The Rise of Quantum Internet Computing',
 'Authors': 'Seng W. Loke',
 'Summary': 'This article highlights quantum Internet computing as referring to\ndistributed quantum computing over the quantum Internet, analogous to\n(classical) Internet computing involving (classical) distributed computing over\nthe (classical) Internet. Relevant to quantum Internet computing would be areas\nof study such as quantum protocols for distributed nodes using quantum\ninformation for computations, quantum cloud computing, delegated verifiable\nblind or private computing, non-local gates, and distributed quantum\napplications, over Internet-scale distances.'}

In [19]:
docs[0].page_content[:10000]  # all pages of the Document content

'arXiv:2208.00733v1  [cs.ET]  1 Aug 2022\nIEEE IOT MAGAZINE, VOL. XX, NO. X, X 2022\n1\nThe Rise of Quantum Internet Computing\nSeng W. Loke, Member, IEEE\nAbstract—This article highlights quantum Internet computing as referring to distributed quantum computing over the quantum Internet,\nanalogous to (classical) Internet computing involving (classical) distributed computing over the (classical) Internet. Relevant to\nquantum Internet computing would be areas of study such as quantum protocols for distributed nodes using quantum information for\ncomputations, quantum cloud computing, delegated veriﬁable blind or private computing, non-local gates, and distributed quantum\napplications, over Internet-scale distances.\nIndex Terms—quantum Internet computing, quantum Internet, distributed quantum computing, Internet computing, distributed\nsystems, Internet\n”This work has been submitted to the IEEE for possible publication. Copyright may be transferred without notice, after which this\nv

### Turns out there's already a semantic search feature for arXiv built into langchain

In [23]:
# get a token: https://platform.openai.com/account/api-keys

from getpass import getpass

OPENAI_API_KEY = getpass()

 ········


In [24]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [27]:
!pip install langchain_openai

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ------------------- -------------------- 20.5/42.0 kB ? eta -:--:--
     ----------------------------- ---------- 30.7/42.0 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 503.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/798.7 kB ? eta -:--:--
   --- ------------------------------------ 61.4/798.7 kB 3.4 MB/s eta 0:00:01
   --- ----------------------------------- 71.7/798.7 kB 991.0 kB/s eta 0:00:01
   ---- ---------------------------------- 92.2/798.7 kB 751.6 kB/s eta 0:00:01
   ---- ---------------------------------- 92.2/798.7 kB 751.6 kB/s eta 0:00:01
   ----------- ---------------------------- 225.3/798.7 kB 1.1 MB/s eta 0:00:01
   -----

In [29]:
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI
from langchain_community.retrievers import ArxivRetriever

retriever = ArxivRetriever(load_max_docs=3)
docs = retriever.get_relevant_documents(query="quantum physics")
docs[0].metadata #meta-information
docs[0].page_content[:400]  # a content of the Document

model = ChatOpenAI(model_name="gpt-3.5-turbo")  # switch to 'gpt-4'
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [30]:
questions = [
    "What are Heat-bath random walks with Markov base?",
    "What is the ImageBind model?",
    "How does Compositional Reasoning with Large Language Models works?",
]
chat_history = []

for question in questions:
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

C:\Users\anmol\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


-> **Question**: What are Heat-bath random walks with Markov base? 

**Answer**: I don't have information on "Heat-bath random walks with Markov base" based on the context provided. 

-> **Question**: What is the ImageBind model? 

**Answer**: The ImageBind model is an approach that learns a joint embedding across six different modalities: images, text, audio, depth, thermal, and IMU data. It demonstrates that only image-paired data is necessary to train such a joint embedding, which binds the modalities together. ImageBind leverages large-scale vision-language models, extending their capabilities to new modalities through natural pairing with images. It enables various applications like cross-modal retrieval, composition of modalities, cross-modal detection and generation. The model sets a new state-of-the-art on zero-shot recognition tasks across modalities and showcases strong few-shot recognition results, outperforming previous work. Additionally, ImageBind serves as a new evaluati